In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re 

In [2]:
def notAPhoto(x):
    return "upload" not in x[2]

def getWikiLink(cell):
    allLinks = list(cell.iterlinks())
    goodLinks = list(filter(lambda x: notAPhoto(x), allLinks))
    if len(goodLinks)==0:
        return "ERROR: No Link"
    else:
        return goodLinks[0][2].replace('/wiki/','')

def getWikiString(string):
    return string.replace('https://en.wikipedia.org/wiki/','')

def getWikiDetails(cells):
    player = cells[0]
    country_cell = cells[1]
    display_name = player.text_content()
    return display_name, getWikiLink(player), getWikiLink(country_cell)

def getFullWikiLink(cell):
    allLinks = list(cell.iterlinks())
    goodLinks = list(filter(lambda x: notAPhoto(x), allLinks))
    if len(goodLinks)==0:
        return "ERROR: No Link"
    else:
        return goodLinks[0][2].replace('/wiki/','https://en.wikipedia.org/wiki/')

def getSquadUrl(url):
    return 'https://en.wikipedia.org/wiki/2019-20_'+ url +'_season'

In [7]:
def validPlayerRow(row):
    skip = False
    goodLength = len(row) in [5,10,12,13,14,]
    if goodLength:
        name_cell = row[3]
        nationality_cell = row[2]
        text = name_cell.text_content().strip()
        skip = text.isnumeric() or len(text) == 0 or text == 'Name' or text == 'Goals' or text == 'Player'
    return goodLength and not skip

def validPlayersTable(table):
    content = table.text_content()
    appsAndGoals = "App" in content and "Goal" in content
    playerOrName = "Player" in content or "Name" in content
    return appsAndGoals and playerOrName

def getPlayers(league,country,teamName,url):
    playerRows = []
    playerWikis = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    allTables = list(doc.xpath('//table'))
    goodTables = list(filter(lambda x: validPlayersTable(x), allTables))
    print("\n",teamName, url)
    if len(goodTables) > 0:
        playerTable = goodTables[0]
        allRows = list(playerTable.xpath('//tr'))
        goodRows = list(filter(lambda x: validPlayerRow(x), allRows))
        for row in goodRows:
            playerCell = row[3]
            playerWiki = getFullWikiLink(playerCell)
            playerName = playerCell.text_content().strip()
            if playerWiki in playerWikis:
                print('Skipping')
            else:
                print(playerName)
    else:
        print('Fucked it for: ',teamName,url)


In [8]:
getPlayers('Prem','England','Chelsea','https://en.wikipedia.org/wiki/2019-20_Chelsea_F.C._season')


 Chelsea https://en.wikipedia.org/wiki/2019-20_Chelsea_F.C._season
Chelsea
Batshuayi  8'
Chelsea
Mount  14'  Emerson  31'  Giroud  67', 88'  Christensen  81'
Chelsea
Chelsea
Abraham  34'  Barkley  81'
Chelsea
Barkley  22'  Kenedy  42'  Mount  57', 60'
Chelsea
Pulisic  20', 28'  Barkley  23' (pen.)  Zouma  41'  Pedro  57'  Batshuayi  88'
Chelsea
Emerson  8'  Kovačić  24'  Abraham  59' (pen.)  Pedro  68'  Giroud  71'  Barkley  86' (pen.)
Reading
Queens Park Rangers
Unknown  ?'
L
Chelsea
Zouma  36'  Jorginho  52'  Abraham  57'  Kanté  90'
Leicester City
Ndidi  67'
Chelsea
Abraham  3', 68'  Mount  17'  Jorginho  33'
Sheffield United
McBurnie  45+1'  Robinson  46'  Zouma  89' (o.g.)
Chelsea
Tomori  31'  Abraham  34', 41', 55',  62'  Christensen  64'  Mount  90+6'
Liverpool
Alexander-Arnold  14',  57'  Firmino  30'  Fabinho  78'  Milner  86'
Brighton & Hove Albion
Maupay  28'  Alzate  33'  Webster  49'
Chelsea
Jorginho  12'  Abraham  17'  Mount  24'  Kanté  40'  Batshuayi  89'
Newcastle Uni

In [9]:
def getTeams(url,league,country):
#     outputData = pd.DataFrame([], columns = ['team','name','country','apps', 'wiki'])
    outputData = []
    page = requests.get(url)
    doc = lh.fromstring(page.content)
    rows = doc.xpath('//tr')
    for row in rows:
        if len(row) == 4:
            cell = row[0]
            teamName = cell.text_content().strip()
            wiki = getWikiLink(cell)
            skip = teamName == 'Team' or teamName == 'Rank' or teamName.isnumeric() or "ERROR" in wiki
            if not skip:
                getPlayers(league,country,teamName,getSquadUrl(wiki))

In [10]:
teams = getTeams('https://en.wikipedia.org/wiki/2019%E2%80%9320_Premier_League','Premier League','England')


 Arsenal https://en.wikipedia.org/wiki/2019-20_Arsenal_F.C._season
Bernd Leno
Héctor Bellerín
Kieran Tierney
Mohamed Elneny
Sokratis Papastathopoulos
Henrikh Mkhitaryan
Dani Ceballos
Alexandre Lacazette
Mesut Özil
Lucas Torreira
Pierre-Emerick Aubameyang
Ainsley Maitland-Niles
Rob Holding
Cédric Soares
Nicolas Pépé
Shkodran Mustafi
Calum Chambers
Pablo  Marí
David Luiz
Reiss Nelson
Emiliano Martínez
Konstantinos Mavropanos
Joe Willock
Matteo Guendouzi
Eddie Nketiah
Sead Kolašinac
Emile Smith Rowe
Matt Macey
Granit Xhaka
Gabriel Martinelli
Bukayo Saka
William Saliba
First team
First team
Bernd Leno
Héctor Bellerín
Kieran Tierney
Sokratis Papastathopoulos
Dani Ceballos
Alexandre Lacazette
Mesut Özil
Lucas Torreira
Pierre-Emerick Aubameyang
Ainsley Maitland-Niles
Rob Holding
Cédric
Nicolas Pépé
Shkodran Mustafi
Calum Chambers
Pablo Marí
David Luiz
Reiss Nelson
Emiliano Martínez
Joe Willock
Matteo Guendouzi
Eddie Nketiah
Sead Kolašinac
Matt Macey
Granit Xhaka
Gabriel Martinelli
Bukayo Sak


 Brighton & Hove Albion https://en.wikipedia.org/wiki/2019-20_Brighton_%26_Hove_Albion_F.C._season
Brighton & Hove Albion
Dunk  11'Murray  32'Locadia  40'Trossard  47'Andone  84'
Brighton & Hove Albion
Richards  84' (pen.)
Brighton & Hove Albion
Groß  25'
Brighton & Hove Albion
Murray  8'Locadia  18'Duffy  73', 90'
Valencia
Rodrigo  28'
W
L
Brighton & Hove Albion
Doucouré  28' (o.g.)Andone  65'Maupay  77'Dunk  88'
West Ham United
Hernández  61'Masuaku  76'Rice  90+4'
Southampton
Ward-Prowse  21'Djenepo  55',  62'Højbjerg  82'Redmond  90+1'
Brighton & Hove Albion
Dunk  83'
Burnley
Lennon  27'Westwood  83'Hendrick  90+1'
Brighton & Hove Albion
Dunk  88'
Brighton & Hove Albion
Maupay  28'Alzate  33'Webster  49'
Tottenham Hotspur
Dier  55'
Brighton & Hove Albion
Webster  21'Mooy  30'  35'Montoya  53'March  87'
Everton
Webster  20' (o.g.)Calvert-Lewin  74'Holgate  86'
Norwich City
Stiepermann  63'
Brighton & Hove Albion
Stephens  5'Dunk  19',  64'Montoya  30'Burn  45+2'Pröpper  74'
Leicest

Southampton
Redmond  22',  74'
Armstrong  48'
Romeu  71'
Sheffield United
Baldock  18'
Guaita  58' (o.g.)
Fleck  68'
Crystal Palace
McCarthy  10'
Benteke  51'
Newcastle United
Bentaleb  21'
Almirón  32'
Lascelles  39'
Ritchie  83'
Joelinton  88'
Lazaro  90+5'
Longstaff  90+6'
Crystal Palace
Benteke  33'
Ayew  70'
Watford
Kiko  31'
Capoue  50'
Doucouré  52'
Pussetto  85'
Crystal Palace
Milivojević  12'
Ayew  23',  45'
Ward  71'
Crystal Palace
Burnley
Tarkowski  58'
Mee  62'
Crystal Palace
McArthur  57'
Chelsea
Giroud  6'
Pulisic  27'
Abraham  71'
Crystal Palace
Zaha  27'
Sakho  27'
Kouyaté  54'
Milivojević  67'
Benteke  90+6'
Manchester United
Wan-Bissaka  36'
Rashford  45+1'
Maguire  65'
Martial  78'
Crystal Palace
McCarthy  64'
Tottenham Hotspur
Kane  13'
Alderweireld  61'
Dier  90+1'
Derby County
Martin  32'
Huddlestone  64'
Malone  90+6'
Colchester United
Wayne Hennessey
Stephen Henderson
Vicente Guaita
Joel Ward
Patrick van Aanholt
James Tomkins
Scott Dann
Mamadou Sakho
Gary Cahill


 Manchester City https://en.wikipedia.org/wiki/2019-20_Manchester_City_F.C._season
West Ham United
Noble  26' (pen.)
Manchester City
Manchester City
D. Silva  13'Sané  40', 55'Sterling  43'Touaizi  80'Pozo  88'
Manchester City
De Bruyne  18'Sterling  40'Nmecha  90+2'
Manchester City
Sterling  12'De Bruyne  33'
L
Manchester City
Gabriel Jesus  25'Sterling  51', 75', 90+1',  90+3'Agüero  86' (pen.)Walker  87'
Tottenham Hotspur
Lamela  23'Lucas  56'
Manchester City
Walker  10'Ederson  13'Agüero  15', 64'Sterling  43'Otamendi  90'
Brighton & Hove Albion
Dunk  83'
Manchester City
Agüero  45'B. Silva  69'Rodri  88'
Watford
Femenía  59'Deulofeu  59'
Manchester City
Gabriel Jesus  24'Rodri  38'Otamendi  51'Mahrez  71'Sterling  84'
Wolverhampton Wanderers
Traoré  80', 90+4'Neves  84'Moutinho  86'
Manchester City
Gabriel Jesus  39'D. Silva  41'Sterling  46'
Aston Villa
Grealish  36'
Southampton
Ward-Prowse  13'Højbjerg  18'
Manchester City
Rodri  65'B. Silva  78'Gabriel Jesus  90+5'
Chelsea
Kan

Southampton
Adams  2'
Southampton
Ings  11', 45' Vokins  83'
Southampton XI
Adams  1' Long  18' Valery  38' Klarer  90+1'
Southampton
Adams  6' Yoshida  34' Boufal  90'
Köln
W
L
Southampton
Liverpool
Mané  45+1'Firmino  71'
Southampton
Djenepo  55'Redmond  90+1'
Manchester United
James  10'
Southampton
Djenepo  66'
Bournemouth
Aké  10'H. Wilson  35'C. Wilson  90+5'
Southampton
Ings  39'
Chelsea
Abraham  17' Mount  24' Kante  40' Batshuayi  89'
Southampton
Ings  53'
Leicester City
Chilwell  10'Tielemans  17'Pérez  19', 39', 57'Vardy  45', 58', 90+3' (pen.)Maddison  85'
Southampton
Ward-Prowse  13'
Everton
Davies  4'Richarlison  75'
Southampton
Ings  8'Ward-Prowse  71'
Watford
Sarr  24'
Norwich City
Pukki  66'
Southampton
Ings  52'
West Ham United
Haller  37'
Southampton
Ings  21', 51'  Stephens  31'
Southampton
Obafemi  31'  Redmond  73'
Crystal Palace
Tomkins  50'
Tottenham Hotspur
Southampton
Armstrong  19'Ings  81'
Wolverhampton Wanderers
Neto  53'Jiménez  65' (pen.), 76'
Southampton


 West Ham United https://en.wikipedia.org/wiki/2019-20_West_Ham_United_F.C._season
West Ham United
Sánchez  6'Antonio  25'Diop  58'
West Ham United
Noble  26' (pen.)
West Ham United
West Ham United
Lanzini  17'
West Ham United
Fornals  11'Lanzini  29'Haller  70'Diangana  78'Antonio  90'
Athletic Bilbao
Haller  1' (o.g.)Williams  15'
L
Manchester City
Gabriel Jesus  25'Sterling  51', 75', 90+1',  90+3'Agüero  86' (pen.)Walker  87'
West Ham United
Hernández  61'Masuaku  76'Rice  90+4'
West Ham United
Noble  3' (pen.)Haller  64', 73'Ogbonna  76'
Norwich City
Aarons  62'
West Ham United
Masuaku  23'  67'Noble  29'
Manchester United
Mata  48'Young  83'
West Ham United
Yarmolenko  10'Cresswell  74'Diop  79'
Crystal Palace
Ward  51'Van Aanholt  63'Ayew  87'Benteke  90+6'
West Ham United
Rice  70'Diop  79'
Sheffield United
Baldock  59'Mousset  69'Stevens  90+2'
Newcastle United
Clark  16'Fernández  22',  90+4'Shelvey  51'
West Ham United
Fredericks  69'
Tottenham Hotspur
Son  36'Lucas Moura  